In [38]:
using LinearAlgebra
using Plots
using MLDatasets
using Statistics
using Random
using Distances
using Distributions
using Roots
using ArgParse
using Clustering
using Combinatorics
using ProgressMeter
using Dates
using Printf
import Sobol: next!
using Sobol
using DataFrames
using CSV
using TensorOperations
using JLD
using JuMP, Ipopt
using Arpack
# import NLopt.optimize!

In [49]:
include("TSNE/myTSNE.jl")
include("kmeans_match_labels.jl")
include("spectral/spectral_clustering.jl")
include("spectral/spectral_reduction_main.jl")
include("spectral/helpers.jl")
include("./datastructs.jl")

In [20]:
df = DataFrame(CSV.File("datasets/abalone.csv", header = 0))
data = convert(Matrix, df[:,2:8]) 
label = convert(Array, df[:, 9]) # 1 ~ 29
k = 29

randseed = 1234; rng = MersenneTwister(randseed)
ind_shuffle = randperm(rng, size(data, 1))
data = data[ind_shuffle, :]
label = label[ind_shuffle]
@info "Size of whole dataset: " size(data), size(label)

n = 4177
# X = data[1:n, :]
# y = label[1:n]
# d = size(X, 2)
# @info "Size of testing data" size(X), size(y)

# # select a fraction of X to be training data
# trainratio = 0.1
# ntrain = Int(floor(n*trainratio))
# Xtrain = X[1:ntrain, :]
# ytrain = y[1:ntrain]
# idtrain = 1:ntrain
# @info "Size of training data" size(Xtrain)

testdata = testingData(data[1:n, :], label[1:n])
X = testdata.X; y = testdata.y
d = testdata.d
@info "Size of testing data" size(X), size(y)
traindata = trainingData(X, y, 0.1)
ntrain = traindata.n
Apm = traindata.Apm
@info "Size of training data" size(traindata.X), size(traindata.y), typeof(traindata)


┌ Info: Size of whole dataset: 
│   (size(data), size(label)) = ((4177, 7), (4177,))
└ @ Main In[20]:10
┌ Info: Size of testing data
│   (size(X), size(y)) = ((4177, 7), (4177,))
└ @ Main In[20]:29
┌ Info: Size of training data
│   (size(traindata.X), size(traindata.y), typeof(traindata)) = ((417, 7), (417,), trainingData)
└ @ Main In[20]:33


In [18]:
# compute Vhat
# @info "Start computing Vhat"
# before = Dates.now()
# Vhat, I_rows = comp_Vhat(X, k, rangeθ; N_sample = 1000) 
# m = size(Vhat, 2)
# @assert m > k 
# after = Dates.now()
# elapsedmin = round(((after - before) / Millisecond(1000))/60, digits=5)
# @info "Vhat size, time cost", size(Vhat), elapsedmin
# Vhat_set = (Vhat = Vhat, rangeθ = rangeθ, I_rows = I_rows, N_sample = 1000, timecost = elapsedmin)
# save("Vhat_set1.jld", "data", Vhat_set)


# or load Vhat
Vhat_set = load("./abalone/saved_data/Vhat_set_false_5_2.jld")["data"]
Vhat = Vhat_set.Vhat
timecost = Vhat_set.timecost
I_rows = Vhat_set.I_rows
m = size(Vhat, 2)
@info "size of Vhat and time cost" size(Vhat), timecost

rangeθ = Vhat_set.rangeθ
dimθ = size(rangeθ, 1)
ntotal = size(X)
@info ntotal, dimθ
@info rangeθ

┌ Info: size of Vhat and time cost
│   (size(Vhat), timecost) = ((4177, 785), 29.48315)
└ @ Main In[18]:20
┌ Info: ((4177, 7), 7)
└ @ Main In[18]:25
┌ Info: [1.0 1000.0; 1.0 1000.0; 1.0 1000.0; 1.0 1000.0; 1.0 1000.0; 1.0 1000.0; 1.0 1000.0]
└ @ Main In[18]:26


In [24]:
loss(θ::Vector) = loss_fun_reduction(X, k, θ, traindata, Vhat; if_deriv = false)[1] 
loss_deriv(θ::Vector) = loss_fun_reduction(X, k, θ, traindata, Vhat)[2] 
function loss7(θ...)
    θ7 = [θ[1], θ[2], θ[3], θ[4], θ[5], θ[6], θ[7]]
    return loss(θ7)
end
function loss7_deriv(θ...)
    θ7 = [θ[1], θ[2], θ[3], θ[4], θ[5], θ[6], θ[7]]
    return loss_deriv(θ7)
end

loss7_deriv (generic function with 1 method)

In [22]:
θ_init = rand(dimθ) .* (rangeθ[:, 2] .- rangeθ[:, 1]) .+ rangeθ[:, 1]

7-element Array{Float64,1}:
 609.9052237428551
 280.05967276308314
 199.03776196486814
 943.1739006535555
 782.0655417681371
 960.9868479268466
 996.1885293530976

In [47]:
@time loss(θ_init)

┌ Info: Evaluate loss func, current θ
│   θ = [609.9052237428551, 280.05967276308314, 199.03776196486814, 943.1739006535555, 782.0655417681371, 960.9868479268466, 996.1885293530976]
└ @ Main /home/xz584/projects/learning_to_cluster/src/spectral/spectral_reduction_main.jl:150


 22.236551 seconds (2.06 M allocations: 672.513 MiB)


-774.7439918937022

In [50]:
@time loss_deriv(θ_init)

┌ Info: Evaluate loss func, current θ
│   θ = [609.9052237428551, 280.05967276308314, 199.03776196486814, 943.1739006535555, 782.0655417681371, 960.9868479268466, 996.1885293530976]
└ @ Main /home/xz584/projects/learning_to_cluster/src/spectral/spectral_reduction_main.jl:150


 50.266831 seconds (2.29 M allocations: 10.897 GiB, 0.61% gc time)


7-element Array{Float64,1}:
 -799.6439628211542
 -799.6386053629052
 -799.6365216929555
 -799.641459005841
 -799.6930131117446
 -799.6795055593509
 -799.660798391816

In [19]:
# use JUMP
model = Model(Ipopt.Optimizer)
# set_optimizer(model, NLopt.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

In [20]:
register(model, :loss7, 7, loss7, loss7_deriv)

In [21]:
@variable(model, 0.1 <= θ[1:7] <= 1500.)

7-element Array{VariableRef,1}:
 θ[1]
 θ[2]
 θ[3]
 θ[4]
 θ[5]
 θ[6]
 θ[7]

In [22]:
@NLobjective(model, Min, loss7(θ...))

In [23]:
@time optimize!(model)

┌ Info: Evaluate loss func, current θ
│   θ = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
└ @ Main /home/xz584/projects/learning_to_cluster/src/spectral/spectral_reduction_main.jl:141
┌ Info: Evaluate loss func, current θ
│   θ = Any[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
└ @ Main /home/xz584/projects/learning_to_cluster/src/spectral/spectral_reduction_main.jl:141


This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0



MethodError: MethodError: no method matching WeightedSqEuclidean(::Array{Any,1})
Closest candidates are:
  WeightedSqEuclidean(!Matched::W) where W<:(AbstractArray{T,N} where N where T<:Real) at /home/xz584/.julia/packages/Distances/jwhuc/src/wmetrics.jl:17

In [21]:
θ_init = rand(Uniform(rangeθ[1], rangeθ[2]), dimθ)

1-element Array{Float64,1}:
 99.00395983379315

In [22]:
# use Optim
before = Dates.now()
results = Optim.optimize(loss, rangeθ[1], rangeθ[2])
θ = Optim.minimizer(results)
after = Dates.now()
elapsedmin = round(((after - before) / Millisecond(1000))/60, digits=5)
@info "Trained θ, time cost " θ, elapsedmin

┌ Info: Evaluate loss func, current θ
│   θ = 76.455005648896
└ @ Main /home/xz584/projects/learning_to_cluster/src/spectral/spectral_reduction_main.jl:141
┌ Info: Evaluate loss func, current θ
│   θ = 123.64499435110397
└ @ Main /home/xz584/projects/learning_to_cluster/src/spectral/spectral_reduction_main.jl:141
┌ Info: Evaluate loss func, current θ
│   θ = 47.28998870220795
└ @ Main /home/xz584/projects/learning_to_cluster/src/spectral/spectral_reduction_main.jl:141
┌ Info: Evaluate loss func, current θ
│   θ = 29.265016946688057
└ @ Main /home/xz584/projects/learning_to_cluster/src/spectral/spectral_reduction_main.jl:141
┌ Info: Evaluate loss func, current θ
│   θ = 18.1249717555199
└ @ Main /home/xz584/projects/learning_to_cluster/src/spectral/spectral_reduction_main.jl:141
┌ Info: Evaluate loss func, current θ
│   θ = 26.805629573671943
└ @ Main /home/xz584/projects/learning_to_cluster/src/spectral/spectral_reduction_main.jl:141
┌ Info: Evaluate loss func, current θ
│   θ = 26.921

In [27]:
L, _ = laplacian_L(X, θ; if_deriv=false) 
H = I_rows == nothing ?  Vhat' * L * Vhat : (Vhat[I_rows, :])' * (L[I_rows, ] * Vhat)
@assert size(H) == (m, m) 

In [30]:
# compute Y, k largest eigenvectors of H
ef = eigen(Symmetric(H), m-k+1:m)
Y = ef.vectors
@info size(Y)

┌ Info: (156, 29)
└ @ Main In[30]:4


In [31]:
a =  kmeans_reduction(Vhat, Y, k; maxiter = 200)
@info size(a)

  Iters               objv        objv-change | affected 
-------------------------------------------------------------
      0       1.644388e+01
      1       1.186057e+01      -4.583318e+00 |       26
      2       1.099561e+01      -8.649586e-01 |       25
      3       1.084326e+01      -1.523509e-01 |       18
      4       1.078773e+01      -5.552359e-02 |       15
      5       1.076699e+01      -2.074553e-02 |       13
      6       1.075254e+01      -1.444404e-02 |       12
      7       1.073694e+01      -1.560151e-02 |       13
      8       1.072220e+01      -1.474449e-02 |       15
      9       1.070162e+01      -2.058171e-02 |       14
     10       1.068273e+01      -1.889056e-02 |       15
     11       1.066043e+01      -2.229761e-02 |       14
     12       1.064022e+01      -2.021263e-02 |       11
     13       1.062779e+01      -1.242714e-02 |       12
     14       1.061785e+01      -9.939768e-03 |       10
     15       1.060827e+01      -9.574211e-03 |       1

┌ Info: (4177,)
└ @ Main In[31]:2


In [32]:
max_acc, matched_assignment = bipartite_match_labels(a, y, k; ntrain = ntrain)
max_acc

0.19946808510638298

In [33]:
RI = randindex(matched_assignment[ntrain+1:end], y[ntrain+1:end])

(0.058294958412671585, 0.8449332948441471, 0.1550667051558529, 0.6898665896882942)